<a href="https://www.kaggle.com/code/owaiskhan9654/granite-4-0-fine-tuning-on-finance-qa-benchmark?scriptVersionId=290100275" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>



<p align="center">
  <img src="https://github.com/Owaiskhan9654/Granite-4.0-Fine-Tuning/blob/main/ibm-granite-4-0-release.jpeg?raw=true" 
       alt="Granite 4.0"
       width="450">
</p>

# **Install dependencies**

In [1]:
%%capture

import os, sys

os.environ["XLA_FLAGS"] = "--xla_cpu_enable_fast_math=false"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"   # also suppresses XLA logs
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

sys.stderr = open(os.devnull, "w")

In [2]:
!pip install -q --no-deps transformers accelerate datasets peft evaluate sentencepiece safetensors bitsandbytes protobuf==3.20.1 mlflow dagshub appdirs databricks-agents huggingface_hub anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 29.5 MB/s eta 0:00:00


In [3]:
%%capture


import json
import torch
import dagshub
import mlflow
import warnings

import pandas as pd
import sys, os, json
from pydantic._internal._generate_schema import UnsupportedFieldAttributeWarning
warnings.filterwarnings("ignore", category=UnsupportedFieldAttributeWarning)
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, create_repo
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling,TrainerCallback


from peft import LoraConfig, get_peft_model
from peft import PeftModel

E0000 00:00:1767595680.236770      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767595680.290721      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **Configurations**

In [4]:
# -----------------------
# Model trainings config
# -----------------------
MODEL_NAME = "ibm-granite/granite-4.0-micro"
DATA_PATH = "/kaggle/working/data.jsonl"   # upload your jsonl here or change path
OUTPUT_DIR = "/kaggle/working/granite-finetuned"
TRAIN_BATCH_SIZE = 4
GRAD_ACCUM = 16
EPOCHS = 5
LEARNING_RATE = 2e-4
MAX_LENGTH = 1024

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
user_secrets = UserSecretsClient()


os.environ["DAGSHUB_USERNAME"] = "owais0425"
os.environ["DAGSHUB_TOKEN"] = user_secrets.get_secret("DAGSHUB_TOKEN")# created from https://dagshub.com/user/settings/tokens
os.environ["DAGSHUB_API_USE_OAUTH"] = "false"
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow"


mlflow_token = user_secrets.get_secret("DAGSHUB_TOKEN")
dagshub.auth.add_app_token(mlflow_token)   # stores token in client cache 
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

In [6]:
dagshub.init(repo_owner="owais0425",repo_name="Granite-4.0-Fine-Tuning",mlflow=True)

Accessing as owais0425

Initialized MLflow to track repo "owais0425/Granite-4.0-Fine-Tuning"

Repository owais0425/Granite-4.0-Fine-Tuning initialized!

In [7]:
class MlflowCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return
        step = getattr(state, "global_step", None) or 0
        for k, v in logs.items():
            # only log numeric metrics
            try:
                val = float(v)
            except Exception:
                continue
            mlflow.log_metric(k, val, step=step)

# **Data Preperations**

### **Example data format (instruction tuning)**


* {"instruction":"Summarize the following: ...", "input":"Long document ...", "output":"Short summary..."}
* {"instruction":"Translate to French", "input":"Hello", "output":"Bonjour"}


In [8]:

CSV_PATH = "/kaggle/input/finder-finance-qa-benchmark-dataset-prep/FinDER-Finance-QA-Benchmark.csv"
JSONL_PATH = "/kaggle/working/FinDER_Finance_QA_Benchmark_processed_instruct_dataset.jsonl"


if os.path.exists(CSV_PATH):
    print("CSV found, converting to JSONL...")

    df = pd.read_csv(CSV_PATH)

    with open(JSONL_PATH, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            # if _==130:
            #     continue

            if _==512:
                break
            obj = {
                # "instruction": row.get("instruction", ""),
                "input": row.get("question/query", ""),
                "output": row.get("answer", "")
            }
            f.write(json.dumps(obj, ensure_ascii=True) + "\n")

    DATA_PATH = JSONL_PATH
    print("Converted JSONL saved at:", DATA_PATH)


else:
    print("CSV not found — creating fallback JSONL...")

    fallback = [
        {"instruction": "Summarize the text.", "input": "Kaggle free GPUs are T4s.", "output": "Kaggle provides free T4 GPUs with limits."},
        {"instruction": "Translate to French.", "input": "Hello", "output": "Bonjour"},
    ]

    fallback_path = "/kaggle/working/fallback_data.jsonl"
    with open(fallback_path, "w", encoding="utf-8") as f:
        for ex in fallback:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")

    DATA_PATH = fallback_path
    print("Using fallback JSONL:", DATA_PATH)

print("\nLoading dataset:", DATA_PATH)

records = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        records.append(json.loads(line))

print("Loaded examples:", len(records))
print("Sample example:", records[0])


CSV found, converting to JSONL...
Converted JSONL saved at: /kaggle/working/FinDER_Finance_QA_Benchmark_processed_instruct_dataset.jsonl

Loading dataset: /kaggle/working/FinDER_Finance_QA_Benchmark_processed_instruct_dataset.jsonl
Loaded examples: 512
Sample example: {'input': 'Delta in CBOE Data & Access Solutions rev from 2021-23.', 'output': 'The Data and Access Solutions revenue increased by $111.5 million from 2021 to 2023, calculated as 539.2 million minus 427.7 million.'}


# **Tokenizer & dataset loading**

In [9]:
import os, json
import pandas as pd
from datasets import load_dataset  # optional, install if missing

CSV_PATH = "/kaggle/input/finder-finance-qa-benchmark-dataset-prep/FinDER-Finance-QA-Benchmark.csv"
JSONL_PATH = "/kaggle/working/FinDER_Finance_QA_Benchmark_processed_instruct_dataset.jsonl"
JSONL_FIXED = "/kaggle/working/FinDER_Finance_QA_Benchmark_processed_instruct_dataset_fixed.jsonl"

df = pd.read_csv(CSV_PATH, keep_default_na=False)
# print("loaded:", df.shape)

bad_idxs = []
for i, val in df['answer'].items():
    if not isinstance(val, str):
        bad_idxs.append((i, type(val).__name__, val))
# show a few
print("non-string 'answer' rows (first 10):", bad_idxs[:10])

with open(JSONL_FIXED, "w", encoding="utf-8") as fw, open(JSONL_PATH, "w", encoding="utf-8") as f_orig:
    bad_logged = 0
    for i, row in df.iterrows():
        
        if i==1000:
            break
        inp = row.get("question/query") or row.get("question") or row.get("text") or ""
        out = row.get("answer", "")
        if isinstance(out, (list, dict)):
            out_str = json.dumps(out, ensure_ascii=False)  # convert list/dict to JSON string
        else:
            out_str = "" if pd.isna(out) else str(out)
        obj = {"input": str(inp), "output": out_str}
        line = json.dumps(obj, ensure_ascii=False)
        fw.write(line + "\n")
        # also write original-style jsonl if you want previous behavior (optional)
        f_orig.write(line + "\n")

print("Wrote fixed JSONL:", JSONL_FIXED)

print("\nValidating with pandas:")
try:
    df_check = pd.read_json(JSONL_FIXED, lines=True)
    print("pandas load OK:", df_check.shape)
except Exception as e:
    print("pandas failed:", e)

print("\nValidating with datasets (pyarrow JSON parser):")
try:
    ds = load_dataset("json", data_files={"train": JSONL_FIXED}, split="train")
    print("datasets load OK:", ds.num_rows)
except Exception as e:
    print("datasets failed:", e)


non-string 'answer' rows (first 10): []
Wrote fixed JSONL: /kaggle/working/FinDER_Finance_QA_Benchmark_processed_instruct_dataset_fixed.jsonl

Validating with pandas:
pandas load OK: (1000, 2)

Validating with datasets (pyarrow JSON parser):


Generating train split: 0 examples [00:00, ? examples/s]

datasets load OK: 1000


In [10]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading dataset (jsonl)...")
# Robust loader for jsonlines
raw_ds = load_dataset("json", data_files={"train": DATA_PATH}, split="train")
# Build single text prompt for SFT; adjust template if you prefer a different format.
def build_prompt(example):
    # instr = example.get("instruction", "")
    inp = example.get("input", "")
    out = example.get("output", "")
    prompt = f"### Input:\n{inp}\n\n### Response:\n{out}" #### Instruction:\n{instr}\n\n
    return prompt

raw_ds = raw_ds.map(lambda ex: {"text": build_prompt(ex)}, remove_columns=raw_ds.column_names)

# Tokenize
print("Tokenizing...")
def tokenize_fn(batch):
    toks = tokenizer(batch["text"], truncation=True, max_length=MAX_LENGTH, padding="max_length")
    toks["labels"] = toks["input_ids"].copy()
    
    return toks

train_ds = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"]) 
print("Prepared dataset — examples:", len(train_ds))


Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Prepared dataset — examples: 1000


# **Model load (BitsAndBytesConfig / fallback)**

In [11]:
print("Preparing model load — trying bitsandbytes 4-bit first if available...")
use_bnb = False
quant_config = None
try:
    import bitsandbytes as bnb  # noqa: F401
    from transformers import BitsAndBytesConfig
    if torch.cuda.is_available():
        use_bnb = True
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
    else:
        print("bitsandbytes installed but CUDA not available — using fp16 fallback.")
except Exception as e:
    print("bitsandbytes import failed or not installed — using fp16 fallback. Error:", str(e))

if use_bnb and quant_config is not None:
    print("Loading model with quantization_config (4-bit)...")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", quantization_config=quant_config)
else:
    print("Fallback load: loading model in fp16 on GPU if available (no bitsandbytes).")
    device_map = "auto" if torch.cuda.is_available() else None
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=(torch.float16 if torch.cuda.is_available() else torch.float32), device_map=device_map)

print("Model loaded — device map:", getattr(model, "hf_device_map", getattr(model, "device_map", None)))

Preparing model load — trying bitsandbytes 4-bit first if available...
Loading model with quantization_config (4-bit)...


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded — device map: {'model.embed_tokens': 0, 'lm_head': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 1, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.layers.32': 1, 'model.layers.33': 1, 'model.layers.34': 1, 'model.layers.35': 1, 'model.layers.36': 1, 'model.layers.37': 1, 'model.layers.38': 1, 'model.layers.39': 1, 'model.norm': 1, 'model.rotary_emb': 1}


# **Apply LoRA (PEFT)** (Quantization)

In [12]:
print("Applying LoRA adapters (PEFT)...") # low rank adaptations
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],# "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.config.use_cache = False
print("PEFT/LoRA applied. Trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))


Applying LoRA adapters (PEFT)...
PEFT/LoRA applied. Trainable params: 1310720


# **Training setup & run (Trainer)**

**All trainings experiments metrics in ML flow can be tracked here** - https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2?viewStateShareKey=3894e7dac091113a949e1a0b144bdfbf23f857b1cfb2b6251e919052fe25b155&compareRunsMode=TABLE****

In [13]:
# import gc, torch

# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

In [14]:
# start MLflow run and attach callback

EXPERIMENT_NAME = "granite-4.0-finetuning-v1"
RUN_NAME = f"granite4-micro-bs{TRAIN_BATCH_SIZE}-lr{LEARNING_RATE}-v3"

mlflow.set_experiment(EXPERIMENT_NAME)
EXPERIMENT_TAG = "granite-4.0-finetuning"


with mlflow.start_run(run_name=RUN_NAME):
    mlflow.set_tag("experiment", EXPERIMENT_TAG)
    mlflow.set_tag("mlflow.source.name", "kaggle_notebook_Granite_4_0_Fine_tuningFinDER_Finance_QA_benchmark")
    mlflow.set_tag("mlflow.source.type", "KAGGLE_NOTEBOOK")
    mlflow.set_tag("project", "granite-finetuning")
    
    mlflow.log_param("train_batch_size", TRAIN_BATCH_SIZE)
    mlflow.log_param("grad_accum", GRAD_ACCUM)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("model_name", MODEL_NAME)

    print("Setting up Trainer...")
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=4,          # small per-GPU, use grad acc
        gradient_accumulation_steps=8,  
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,                       # force mixed precision when using GPU
        logging_steps=10,                # reduce logging overhead
        save_strategy="steps",
        save_steps=500,                 
        save_total_limit=3,
        remove_unused_columns=False,
        report_to="none",
        label_names=["labels"],
        dataloader_num_workers=4,     
        dataloader_pin_memory=True,
        # evaluation_strategy="no",
        disable_tqdm=False
    )


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        data_collator=data_collator,
    )

    trainer.add_callback(MlflowCallback())

    print("Starting training — this may take a while depending on data and GPU resources.")
    trainer.train()

    # save & log model artifact
    trainer.save_model(OUTPUT_DIR)
    mlflow.log_artifact(OUTPUT_DIR)


Setting up Trainer...
Starting training — this may take a while depending on data and GPU resources.


Step,Training Loss
10,2.090800
20,1.902200
30,1.865300
40,1.801500
50,1.773600
60,1.748900
70,1.735800
80,1.726400
90,1.714200
100,1.695800


🏃 View run granite4-micro-bs4-lr0.0002-v3 at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2/runs/6b40805e406b4943ad9897fe381a7e06
🧪 View experiment at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2


# **Save adapters + tokenizer**

In [15]:
print("Saving LoRA adapter and tokenizer to:", OUTPUT_DIR)
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved.")

Saving LoRA adapter and tokenizer to: /kaggle/working/granite-finetuned
Saved.


In [16]:
model = model.merge_and_unload()   # returns merged HF model (if using peft>=0.5.x)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [17]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Saved FULL merged model to:", OUTPUT_DIR)

Saved FULL merged model to: /kaggle/working/granite-finetuned


In [18]:
HF_TOKEN = user_secrets.get_secret("Hugging_Face_model_Push_Secret")

REPO_ID = "owaiskha9654/granite-4-finetuned-finance-qa-data"
LOCAL_DIR = OUTPUT_DIR

# # Creating repository if not exists
#create_repo(REPO_ID, token=HF_TOKEN, private=False)

# Uploading entire folder (model + LoRA + tokenizer)
api = HfApi()
api.upload_folder(
    folder_path=LOCAL_DIR,
    repo_id=REPO_ID,
    token=HF_TOKEN,
    commit_message="Upload fine-tuned model and LoRA adapter"
)

print("Uploaded to Hugging Face:", REPO_ID)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Uploaded to Hugging Face: owaiskha9654/granite-4-finetuned-finance-qa-data


# **Quick inference test**

In [19]:
base = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
peft_model = PeftModel.from_pretrained(base, OUTPUT_DIR)
peft_model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GraniteMoeHybridForCausalLM(
      (model): GraniteMoeHybridModel(
        (embed_tokens): Embedding(100352, 2560, padding_idx=100256)
        (layers): ModuleList(
          (0-39): 40 x GraniteMoeHybridDecoderLayer(
            (input_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
            (post_attention_layernorm): GraniteMoeHybridRMSNorm((2560,), eps=1e-05)
            (shared_mlp): GraniteMoeHybridMLP(
              (activation): SiLU()
              (input_linear): Linear(in_features=2560, out_features=16384, bias=False)
              (output_linear): Linear(in_features=8192, out_features=2560, bias=False)
            )
            (self_attn): GraniteMoeHybridAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=2560, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
            

**All trainings experiments metrics in ML flow can be tracked here** - https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/2?viewStateShareKey=3894e7dac091113a949e1a0b144bdfbf23f857b1cfb2b6251e919052fe25b155&compareRunsMode=TABLE

**Its basically a Quantization aware training post Callibration**

In [20]:
prompt = "### Instruction:\nSummarize the text.\n\n### Input:\nKaggle T4 GPU is a powerful tool for data scientists and machine learning enthusiasts. It provides access to Google's Tensor Processing Units (TPUs) for running machine learning models and performing data analysis.\n\n ### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to(next(peft_model.parameters()).device)
out = peft_model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(out[0], skip_special_tokens=True))

### Instruction:
Summarize the text.

### Input:
Kaggle T4 GPU is a powerful tool for data scientists and machine learning enthusiasts. It provides access to Google's Tensor Processing Units (TPUs) for running machine learning models and performing data analysis.

 ### Response:
Kaggle T4 GPU is a specialized cloud-based service that provides access to Google’s Tensor Processing Units (TPUs). These TPUs are designed to accelerate machine learning workloads, making them ideal for data scientists and machine learning enthusiasts who need high-performance computing resources. By leveraging this service, users can run complex machine learning models and perform intensive data analysis tasks more efficiently, benefiting from the advanced processing capabilities of TPUs. This access is particularly valuable for those working on large-scale data processing, model training, and inference, as it allows them to scale their computational resources without the need for on-premises hardware.


# **Evaluation**

LLM Evaluation Metrics
There are two types of LLM evaluation metrics:

**Heuristic-based metrics:** These metrics calculate a score for each data record (row in terms of Pandas/Spark dataframe), based on certain functions, such as: Rouge (mlflow.metrics.rougeL()), Flesch Kincaid (mlflow.metrics.flesch_kincaid_grade_level()) or Bilingual Evaluation Understudy (BLEU) (mlflow.metrics.bleu()). These metrics are similar to traditional continuous value metrics. For the list of built-in heuristic metrics and how to define a custom metric with your own function definition, see the Heuristic-based Metrics section.

**LLM-as-a-Judge metrics:** LLM-as-a-Judge is a new type of metric that uses LLMs to score the quality of model outputs. It overcomes the limitations of heuristic-based metrics, which often miss nuances like context and semantic accuracy. LLM-as-a-Judge metrics provides a more human-like evaluation for complex language tasks while being more scalable and cost-effective than human evaluation. MLflow provides various built-in LLM-as-a-Judge metrics and supports creating custom metrics with your own prompt, grading criteria, and reference examples. See the LLM-as-a-Judge Metrics section for more details.



In [21]:
df

,_id,question/query,reasoning,category,references,answer,type
0,b33fcee7,Delta in CBOE Data & Access Solutions rev from...,True,Financials,"['Cboe Global Markets, Inc. and Subsidiaries\n...",The Data and Access Solutions revenue increase...,Subtract
1,b8a1383c,CBOE CBA expiring may impact union representat...,True,Company overview,"['Employees\n\nAs of December 31, 2023, we emp...",The three building engineers represent approxi...,Compositional
2,9826209b,Cboe's earnings impact from market data vs reg...,False,Footnotes,['North\n\n\n\n\n\n\n\n\n\n\nCorporate\n\n\n\n...,The data provided shows that Cboe Global Marke...,None
3,30eb0cd9,Cboe’s repurchase costs & the impact of its au...,False,Shareholder return,"['In 2011, the Board of Directors approved an ...",The available data on share repurchase activit...,None
4,1a4cebce,"Cboe's operational stability, governance in cy...",False,Risk,"['We maintain policies, procedures and control...",Cboe Global Markets Inc.'s multi-layered cyber...,None
...,...,...,...,...,...,...,...
5698,707224b0,"Execs at Charles Schwab, incl. age & experienc...",False,Governance,['Executive Officers of the Registrant\nName\t...,The provided data allows us to assess the pote...,None
5699,b0fa24d5,"SCHW's revenue recognition methods differ, imp...",False,Accounting,['Net interest revenue\n\nNet interest revenue...,The revenue recognition methods differ conside...,None
5700,037b46e9,Liquidity safety for FDIC insured deposits SCHW.,False,Financials,['Schwab’s primary source of funds is cash gen...,The fact that more than 80% of bank deposits a...,None
5701,5e85a743,capitalized software vs building IT capex for ...,False,Footnotes,['Capital expenditures primarily include capit...,The data indicates that the overall capital ex...,None


In [22]:
CSV_PATH = "/kaggle/input/finder-finance-qa-benchmark-dataset-prep/FinDER-Finance-QA-Benchmark.csv"
JSONL_PATH = "/kaggle/working/FinDER_Finance_QA_Benchmark_eval_groundtruth.jsonl"
START_INDEX = 1200 

assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

rows = []
with open(JSONL_PATH, "w", encoding="utf-8") as fw:
    for idx, row in df.iterrows():
        if idx <= START_INDEX:
            continue
        if idx>1220:
            break
        instr = row.get("instruction", "") or ""
        inp = row.get("question/query", "") or ""
        out = row.get("answer", "") or ""
        # Build a combined prompt input (adjust formatting as needed)
        prompt = instr.strip()
        if inp.strip():
            prompt = prompt + "\n\n" + inp.strip()
        obj = {
            # "instruction": instr,
            "input": inp,
            "output": out,
            # ground-truth format for MLflow / eval usage:
            "inputs": prompt,
            "ground_truth": out
        }
        fw.write(json.dumps(obj, ensure_ascii=True) + "\n")
        rows.append({"inputs": prompt, "ground_truth": out})

print(f"Saved evaluation JSONL to: {JSONL_PATH}")
print(f"Total eval rows written (index >= {START_INDEX}): {len(rows)}")

eval_df = pd.DataFrame(rows)
# eval_df = eval_df[eval_df["ground_truth"].astype(str).str.strip() != ""].reset_index(drop=True)
print(f"Final eval_df rows after dropping empty ground-truths: {len(eval_df)}")

# Show head
eval_df


Saved evaluation JSONL to: /kaggle/working/FinDER_Finance_QA_Benchmark_eval_groundtruth.jsonl
Total eval rows written (index >= 1200): 20
Final eval_df rows after dropping empty ground-truths: 20


,inputs,ground_truth
0,\n\n2023 components contribution for Akamai (A...,"In 2023, the net cash used in investing activi..."
1,\n\nImpact of Akamai purchase commitments on l...,The provided data indicates that Akamai’s long...
2,"\n\nAKAM. Comp. positioning, global mktg, loca...",The provided information explains that Akamai ...
3,\n\nCDNS R&D to Rev ratio FY23 & implications ...,To calculate the ratio of Research and Develop...
4,\n\nCadence’s competitive positioning is influ...,The reference outlines that Cadence Design Sys...
5,\n\nCadence's (CDNS) rev vs asset alloc by reg...,"The available data, which provides revenue and..."
6,"\n\nCDNS share repurchase drivers include biz,...",The disclosure notes that the company’s share ...
7,\n\nImpact of CDNS cert on investor sentiment ...,The implementation of internationally recogniz...
8,\n\nCadence: impact of mixed competition on bi...,The provided reference allows us to analyze qu...
9,\n\nThe impact of litigation on CDD cash flows...,The uncertainties surrounding current disputes...


In [23]:
from anthropic import Anthropic
from mlflow.metrics import latency, toxicity
from mlflow.metrics.genai import answer_correctness


os.environ["ANTHROPIC_API_KEY"] = user_secrets.get_secret("anthropic_api_key") 
os.environ["OPENAI_API_KEY"] = os.environ["ANTHROPIC_API_KEY"]

client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
# mlflow.environment_variables.MLFLOW_GENAI_PROVIDER.set("anthropic")
# mlflow.environment_variables.MLFLOW_GENAI_API_KEY.set(os.environ["ANTHROPIC_API_KEY"])

In [24]:
def anthropic_model(inputs):
    outputs = []
    for q in inputs:
        resp = client.messages.create(
            model="claude-sonnet-4-5-20250929",   
            messages=[{"role": "user", "content": q}],
            max_tokens=512,
            temperature=0.0
        )
    
        try:
            text = resp.content[0].text
        except:
            text = str(resp)
        
        outputs.append(text)
    return outputs

In [25]:
# mlflow.set_experiment("llm-based-evaluation")

EXPERIMENT_NAME = "anthropic-evaluation-v3"
RUN_NAME = f"granite4-anthropic-evaluation-v3"

mlflow.set_experiment(EXPERIMENT_NAME)
EXPERIMENT_TAG = "granite-4.0-finetuning"


with mlflow.start_run(run_name=RUN_NAME):
    mlflow.set_tag("experiment", EXPERIMENT_TAG)
    mlflow.set_tag("mlflow.source.name", "kaggle_notebook_Granite_4_0_Fine_tuning")
    mlflow.set_tag("mlflow.source.type", "KAGGLE_NOTEBOOK")
    mlflow.set_tag("project", "granite-finetuning")

    results = mlflow.evaluate(
        model=anthropic_model,              
        data=eval_df,
        # input_columns="inputs",
        targets="ground_truth",
        model_type="question-answering",
        extra_metrics=[answer_correctness(), latency(),toxicity()],
        evaluator_config={"num_workers": 1}
    )

    print("📊 Aggregated metrics:")
    print(results.metrics)

    print("\n🧾 Evaluation table:")
    display(results.tables["eval_results_table"])

/tmp/ipykernel_20/2488923104.py:22: FutureWarning: ``mlflow.metrics.genai.metric_definitions.answer_correctness`` is deprecated since 3.4.0. Use the new GenAI evaluation functionality instead. See https://mlflow.org/docs/latest/genai/eval-monitor/legacy-llm-evaluation/ for the migration guide.
  extra_metrics=[answer_correctness(), latency(),toxicity()],
/usr/local/lib/python3.11/dist-packages/mlflow/metrics/genai/metric_definitions.py:178: FutureWarning: ``mlflow.metrics.genai.genai_metric.make_genai_metric`` is deprecated since 3.4.0. Use the new GenAI evaluation functionality instead. See https://mlflow.org/docs/latest/genai/eval-monitor/legacy-llm-evaluation/ for the migration guide.
  return make_genai_metric(
/tmp/ipykernel_20/2488923104.py:22: FutureWarning: ``mlflow.metrics.latency`` is deprecated since 3.4.0. Use the new GenAI evaluation functionality instead. See https://mlflow.org/docs/latest/genai/eval-monitor/legacy-llm-evaluation/ for the migration guide.
  extra_metrics=

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

📊 Aggregated metrics:
{'latency/mean': 6.677376294136048, 'latency/variance': 1.2121070511448118, 'latency/p90': 7.31606366634369, 'toxicity/v1/mean': 0.0004450658743735403, 'toxicity/v1/variance': 0.0, 'toxicity/v1/p90': 0.0004450658743735403, 'toxicity/v1/ratio': 0.0, 'exact_match/v1': 0.0, 'answer_correctness/v1/mean': nan, 'answer_correctness/v1/variance': nan}

🧾 Evaluation table:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,inputs,ground_truth,outputs,latency,token_count,toxicity/v1/score,answer_correctness/v1/score,answer_correctness/v1/justification
0,\n\n2023 components contribution for Akamai (A...,"In 2023, the net cash used in investing activi...","I'd be happy to help you with inputs! However,...",6.766314,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
1,\n\nImpact of Akamai purchase commitments on l...,The provided data indicates that Akamai’s long...,"I'd be happy to help you with inputs! However,...",6.313200,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
2,"\n\nAKAM. Comp. positioning, global mktg, loca...",The provided information explains that Akamai ...,"I'd be happy to help you with inputs! However,...",6.272299,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
3,\n\nCDNS R&D to Rev ratio FY23 & implications ...,To calculate the ratio of Research and Develop...,"I'd be happy to help you with inputs! However,...",6.525210,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
4,\n\nCadence’s competitive positioning is influ...,The reference outlines that Cadence Design Sys...,"I'd be happy to help you with inputs! However,...",5.990041,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
5,\n\nCadence's (CDNS) rev vs asset alloc by reg...,"The available data, which provides revenue and...","I'd be happy to help you with inputs! However,...",6.096150,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
6,"\n\nCDNS share repurchase drivers include biz,...",The disclosure notes that the company’s share ...,"I'd be happy to help you with inputs! However,...",6.824123,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
7,\n\nImpact of CDNS cert on investor sentiment ...,The implementation of internationally recogniz...,"I'd be happy to help you with inputs! However,...",6.133199,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
8,\n\nCadence: impact of mixed competition on bi...,The provided reference allows us to analyze qu...,"I'd be happy to help you with inputs! However,...",6.537505,160,0.000445,NaN,Failed to score model on payload. Error: Faile...
9,\n\nThe impact of litigation on CDD cash flows...,The uncertainties surrounding current disputes...,"I'd be happy to help you with inputs! However,...",8.234499,160,0.000445,NaN,Failed to score model on payload. Error: Faile...


🏃 View run granite4-anthropic-evaluation-v3 at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/6/runs/9d48f6cf9f584d3a8d5e958fce8d7ae9
🧪 View experiment at: https://dagshub.com/owais0425/Granite-4.0-Fine-Tuning.mlflow/#/experiments/6


[Trace(trace_id=tr-bdd640fb06671ad11c80317fa3b1799d), Trace(trace_id=tr-1a3d1fa7bc8960a923b8c1e9392456de), Trace(trace_id=tr-972a846916419f828b9d2434e465e150), Trace(trace_id=tr-3b8faa1837f8a88b17fc695a07a0ca6e), Trace(trace_id=tr-b74d0fb132e706298fadc1a606cb0fb3), Trace(trace_id=tr-72ff5d2a386ecbe06b65a6a48b8148f6), Trace(trace_id=tr-c241330b01a9e71fde8a774bcf36d58b), Trace(trace_id=tr-47229389571aa8766c307511b2b9437a), Trace(trace_id=tr-1a2a73ed562b0f79c37459eef50bea63), Trace(trace_id=tr-580d7b71d8f564135be6128e18c26797)]